In [2]:
!pip install tensorflow numpy opencv-python gradio langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machin

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt
import cv2
import gradio as gr
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

model = load_model('/content/drive/MyDrive/fused_model.h5')

class_names = [
    "Grape___Black_rot",
    "Grape__Esca(Black_Measles)",
    "Grape__Leaf_blight(Isariopsis_Leaf_Spot)",
    "Grape___healthy"
]

def predict_disease(img):
    """Predict the disease from the uploaded grape leaf image"""
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (256, 256))  # Resize to 256x256
    img_array = np.expand_dims(img_resized, axis=0)
    img_array = preprocess_input(img_array)
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    predicted_class = class_names[predicted_class_index]

    return predicted_class, cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR)

def clean_llama_output(text):
    """Clean the LLaMA output by removing metadata and formatting the content"""
    if "additional_kwargs=" in text:
        text = text.split("additional_kwargs=")[0]

    if 'content="' in text:
        text = text.replace('content="', '')

    if 'content=' in text:
        text = text.replace('content=', '')

    if "'" in text:
        text = text.replace("'", '')


    text = text.replace('\\n\\n', '\n\n')


    text = text.replace('\\n', '\n')


    text = text.strip('"')

    return text

def format_measures(disease_name, measures_text):
    """Format the measures into properly structured markdown"""

    clean_text = clean_llama_output(measures_text)

    markdown_content = f"""
# Disease Analysis Report

## Condition Detected
{disease_name.replace('', ' ').replace('_', ' ')}

## Analysis and Recommendations
{clean_text}

---
Note: Please consult with a local agricultural expert before implementing any treatments.
"""
    return markdown_content

def predict_measure(predict_class):
    prompt_template = PromptTemplate.from_template(
        "Analyze the following grape leaf condition and provide a detailed report."
        "\nCondition: {detected_condition}"
        "\n\nProvide your response in the following format:"
        "\n1. Brief Description: Describe the condition and its cause"
        "\n2. Treatment Measures: List specific treatment steps"
        "\n3. Preventive Measures: List prevention strategies"
        "\n4. Monitoring Guidelines: Describe monitoring practices"
        "\n\nFormat each section with clear headers."

    )

    formatted_prompt = prompt_template.format(detected_condition=predict_class)

    model = ChatGroq(
        temperature=0,
        model_name="llama-3.1-8b-instant",
        api_key="groq_api_key"
    )

    output = model.invoke(formatted_prompt)
    return format_measures(predict_class, str(output))

# Gradio interface
def gradio_interface(img):
    predicted_class, processed_image = predict_disease(img)
    measures = predict_measure(predicted_class)

    # Format the prediction text
    prediction_text = f"""
### Detected Condition
{predicted_class.replace('', ' ').replace('_', ' ')}
"""

    return processed_image, prediction_text, measures

interface = gr.Blocks(theme=gr.themes.Soft())

with interface:
    gr.Markdown("""
    # 🍇 Grape Leaf Disease Detection System

    This advanced tool uses deep learning to identify diseases in grape leaves and provide actionable recommendations.
    Upload a clear image of a grape leaf for analysis and treatment recommendations.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Upload Image")
            image_input = gr.Image(
                type="numpy",
                label="Upload Grape Leaf Image",
                elem_id="image_input"
            )
            predict_button = gr.Button(
                "Analyze Leaf",
                variant="primary",
                size="lg",
                interactive=True
            )

        with gr.Column(scale=2):
            gr.Markdown("### Analysis Results")
            with gr.Row():
                image_output = gr.Image(
                    type="numpy",
                    label="Processed Image",
                    interactive=False,
                    show_download_button=True
                )
            with gr.Row():
                prediction_output = gr.Markdown()
            with gr.Row():
                measures_output = gr.Markdown()

    gr.Markdown("""
    ### Usage Guidelines
    - Upload a clear, well-lit image of the grape leaf
    - Ensure the leaf is centered and in focus
    - For best results, capture the entire leaf in the image
    """)

    predict_button.click(
        fn=gradio_interface,
        inputs=image_input,
        outputs=[image_output, prediction_output, measures_output],
        api_name="predict",
        show_progress=True,
    )


interface.queue().launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d695a1d72c01dfdb9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
